In [23]:
# Importing necessary libraries
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import random

# Define the number of qubits
n_qubits = 3

# Initialize a device
dev = qml.device('default.qubit', wires=n_qubits, shots=1)

# Define the initial state preparation function
def initial_state():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])

# Define the weak error function
def apply_weak_error():
    # Randomly select one of the three qubits
    qubit_to_flip = random.randint(0, 2)
    # Apply a small rotation around the X axis to simulate a weak bit-flip error
    epsilon = np.pi /3
    qml.RX(epsilon, wires=qubit_to_flip)

def plot_circuit_and_probs(circuit, *args, description=""):
    # Execute the circuit to obtain the probabilities
    probs = circuit(*args)
    # Calculate the squared magnitude of each probability amplitude
    squared_probs = np.abs(probs)**2
    # Draw the circuit
    fig, ax = qml.draw_mpl(circuit, expansion_strategy="device")(*args)
    plt.show()
    # Print the probabilities
    print(f"Probabilities of states after {description}:")
    for i, p in enumerate(squared_probs):
        print(f"State |{i:03b}⟩: {p:.4f}")
    print("\n")

@qml.qnode(dev)
def circuit():
    initial_state()
    apply_weak_error()
    
    # Measure the stabilizers
    Z1Z2 = qml.sample(qml.PauliZ(0) @ qml.PauliZ(1))
    Z1Z3 = qml.sample(qml.PauliZ(0) @ qml.PauliZ(2))
    
    # Define error correction based on the stabilizer measurements
    if Z1Z2 == -1 and Z1Z3 == -1:
        qml.PauliX(wires=0)
    elif Z1Z2 == -1 and Z1Z3 == 1:
        qml.PauliX(wires=1)
    elif Z1Z2 == 1 and Z1Z3 == -1:
        qml.PauliX(wires=2)
    
    
    return qml.probs(wires=[0, 1, 2])




QuantumFunctionError: All measurements must be returned in the order they are measured.